In [ ]:
#Install prerequisits
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 torchtext --extra-index-url https://download.pytorch.org/whl/cu116
!pip install pymc arviz chex cvxpy flax librosa pydantic tweepy fairscale hydra-core omegaconf pandas wandb==0.12.21 pytorch_lightning==1.5.10 biopython==1.79 ml-collections==0.1.0 einops typing-extensions==3.10.0.2 deepspeed==0.5.9 dm-tree==0.1.6 numpy==1.21.2 PyYAML==5.4.1 requests scipy==1.7.1 tqdm==4.62.2
!pip install git+https://github.com/NVIDIA/dllogger.git

!pip install fair-esm
!pip install "fair-esm[esmfold]"

# OpenFold and its remaining dependency
!pip install 'dllogger @ git+https://github.com/NVIDIA/dllogger.git'
!pip install 'openfold @ git+https://github.com/aqlaboratory/openfold.git@4b41059694619831a7db195b7e0988fc4ff3a307'

In [ ]:
#Access google drive
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/MO_Comp_Genomics/esmfold')

In [ ]:
#read fasta file
from Bio import SeqIO

fasta_file = "rerun.fasta"

records = list(SeqIO.parse(fasta_file, "fasta"))

# Sort the sequences by length
records_sorted = sorted(records, key=lambda x: len(x.seq), reverse=True )

In [ ]:
import torch
import esm

model = esm.pretrained.esmfold_v1()
model = model.eval().cuda()

In [ ]:
#Iterate each record and predict the structure
model.set_chunk_size(256)
for i, record in enumerate(records_sorted):
  seqID  = str(record.id)
  seqLen = len(str(record.seq))

  #check if the protein structure exists
  if f'{seqID}.pdb' not in os.listdir():
    if seqLen <= 1000:
      with torch.no_grad():
        output = model.infer_pdb(str(record.seq))

      with open(f'{seqID}.pdb', 'w') as f:
        f.write(output)
      torch.cuda.empty_cache()